## 长短期记忆（LSTM）— 从0开始

上一节中，我们介绍了循环神经网络中的梯度计算方法。我们发现，循环神经网络的隐含层变量梯度可能会出现衰减或爆炸。虽然梯度裁剪可以应对梯度爆炸，但无法解决梯度衰减的问题。因此，给定一个时间序列，例如文本序列，循环神经网络在实际中其实较难捕捉两个时刻距离较大的文本元素（字或词）之间的依赖关系。

为了更好地捕捉时序数据中间隔较大的依赖关系，我们介绍了一种常用的门控循环神经网络，叫做门控循环单元。本节将介绍另一种常用的门控循环神经网络，长短期记忆（long short-term memory，简称LSTM）。它由Hochreiter和Schmidhuber在1997年被提出。事实上，它比门控循环单元的结构稍微更复杂一点。

### 长短期记忆
我们先介绍长短期记忆的构造。长短期记忆的隐含状态包括隐含层变量H和细胞C（也称记忆细胞）。它们形状相同。

### 输入门、遗忘门和输出门

假定隐含状态长度为h，给定时刻$t$的一个样本数为$n$特征向量维度为x的批量数据$\mathbf{X}_t \in \mathbb{R}^{n \times x}$和上一时刻隐含状态$\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$，输入门（input gate）$\mathbf{I}_t \in \mathbb{R}^{n \times h}$、遗忘门（forget gate）$\mathbf{F}_t \in \mathbb{R}^{n \times h}$和输出门（output gate）$\mathbf{O}_t \in \mathbb{R}^{n \times h}$的定义如下：

$\mathbf{I}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xi} + \mathbf{H}_{t-1} \mathbf{W}_{hi} + \mathbf{b}_i)$

$\mathbf{F}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xf} + \mathbf{H}_{t-1} \mathbf{W}_{hf} + \mathbf{b}_f)$

$\mathbf{O}_t = \sigma(\mathbf{X}_t \mathbf{W}_{xo} + \mathbf{H}_{t-1} \mathbf{W}_{ho} + \mathbf{b}_o)$

其中的$\mathbf{W}_{xi}, \mathbf{W}_{xf}, \mathbf{W}_{xo} \in \mathbb{R}^{x \times h}$和$\mathbf{W}_{hi}, \mathbf{W}_{hf}, \mathbf{W}_{ho} \in \mathbb{R}^{h \times h}$是可学习的权重参数，$\mathbf{b}_i, \mathbf{b}_f, \mathbf{b}_o \in \mathbb{R}^{1 \times h}$是可学习的偏移参数。函数$\sigma$自变量中的三项相加使用了广播。

和门控循环单元中的重置门和更新门一样，这里的输入门、遗忘门和输出门中每个元素的值域都是$[0,1]$。

### 候选细胞
和门控循环单元中的候选隐含状态一样，长短期记忆中的候选细胞$\tilde{\mathbf{C}}_t \in \mathbb{R}^{n \times h}$也使用了值域在[−1,1]的双曲正切函数$\tanh$做激活函数：

$\tilde{\mathbf{C}}_t = \text{tanh}(\mathbf{X}_t \mathbf{W}_{xc} + \mathbf{H}_{t-1} \mathbf{W}_{hc} + \mathbf{b}_c)$

其中的$\mathbf{W}_{xc} \in \mathbb{R}^{x \times h}$和$\mathbf{W}_{hc} \in \mathbb{R}^{h \times h}$是可学习的权重参数，$\mathbf{b}_c \in \mathbb{R}^{1 \times h}$是可学习的偏移参数。

### 细胞
我们可以通过元素值域在$[0,1]$的输入门、遗忘门和输出门来控制隐含状态中信息的流动：这通常可以应用按元素乘法符$\odot$。当前时刻细胞$\mathbf{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时刻细胞和当前时刻候选细胞的信息，并通过遗忘门和输入门来控制信息的流动：

$\mathbf{C}_t = \mathbf{F}_t \odot \mathbf{C}_{t-1} + \mathbf{I}_t \odot \tilde{\mathbf{C}}_t$

需要注意的是，如果遗忘门一直近似1且输入门一直近似0，过去的细胞将一直通过时间保存并传递至当前时刻。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时序数据中间隔较大的依赖关系。

### 隐含状态
有了细胞以后，接下来我们还可以通过输出门来控制从细胞到隐含层变量$\mathbf{H}_t \in \mathbb{R}^{n \times h}$的信息的流动：

$\mathbf{H}_t = \mathbf{O}_t \odot \text{tanh}(\mathbf{C}_t)$

需要注意的是，当输出门近似$1$，细胞信息将传递到隐含层变量；当输出门近似$0$，细胞信息只自己保留。

输出层的设计可参照循环神经网络中的描述。

### 实验
为了实现并展示门控循环单元，我们依然使用周杰伦歌词数据集来训练模型作词。这里除长短期记忆以外的实现已在循环神经网络中介绍。

### 数据处理
我们先读取并对数据集做简单处理。



In [1]:
import zipfile
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip', 'r') as zin:
    zin.extractall('../../data/')

with open('../../data/jaychou_lyrics.txt') as f:
    corpus_chars = f.read().decode('utf-8')

corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:20000]

idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
corpus_indices = [char_to_idx[char] for char in corpus_chars]

vocab_size = len(char_to_idx)
print('vocab size:', vocab_size)

('vocab size:', 1465)


我们使用onehot来将字符索引表示成向量。



In [2]:
import numpy as np

def one_hot(positions, vocal_size):
    if len(positions.shape) == 0:
        positions = np.expand_dims(positions, 0)
        positions = np.expand_dims(positions, 1)
    if len(positions.shape) == 1:
        positions = np.expand_dims(positions, 1)
    all_zeros = np.zeros((positions.shape[0], vocab_size))
    for i in xrange(positions.shape[0]):
        for j in xrange(positions.shape[1]):
            all_zeros[i, int(positions[i, j])] = 1
        
    return all_zeros
        
one_hot(np.array([0, 2]), vocab_size)

def get_inputs(data):
    return [one_hot(X, vocab_size) for X in data.T]


### 初始化模型参数
以下部分对模型参数进行初始化。参数hidden_dim定义了隐含状态的长度。



In [3]:
import random
import numpy as np

def data_iter_random(corpus_indices, batch_size, num_steps):
    # 减一是因为label的索引是相应data的索引加一
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    # 随机化样本
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回num_steps个数据
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        data = np.array(
            [_data(j * num_steps) for j in batch_indices])
        label = np.array(
            [_data(j * num_steps + 1) for j in batch_indices])
        yield data.astype(np.float32), label

In [4]:
import tensorflow as tf

input_dim = vocab_size
# 隐含状态长度
hidden_dim = 256
output_dim = vocab_size
weight_scale = .01


with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
    # input gate
    W_xi = tf.get_variable(name='weights_xi', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hi = tf.get_variable(name='weights_hi', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_i = tf.get_variable(name='bias_i', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)

    # forget gate
    W_xf = tf.get_variable(name='weights_xf', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hf = tf.get_variable(name='weights_hf', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_f = tf.get_variable(name='bias_f', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    
    # output gate
    W_xo = tf.get_variable(name='weights_xo', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_ho = tf.get_variable(name='weights_ho', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_o = tf.get_variable(name='bias_o', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
                          
    # candidate
    W_xc = tf.get_variable(name='weights_xc', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    W_hc = tf.get_variable(name='weights_hc', shape=[hidden_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_c = tf.get_variable(name='bias_c', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
    
    W_hy = tf.get_variable(name='weights_y', shape=[hidden_dim, output_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_y = tf.get_variable(name='bias_y', shape=[output_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)


params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hy, b_y]


### 定义模型
我们将前面的模型公式翻译成代码。

In [5]:
def lstm_rnn(inputs, state_h, state_c, *params):
    # inputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵
    # H: 尺寸为 batch_size * hidden_dim 矩阵
    # outputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
     W_hy, b_y] = params[0]

    H = state_h
    C = state_c
    num_steps = inputs.get_shape().as_list()[0]
    outputs = []
    for i in range(num_steps):
        X = inputs[i]
        I = tf.nn.sigmoid(tf.matmul(X, W_xi) + tf.matmul(H, W_hi) + b_i)
        F = tf.nn.sigmoid(tf.matmul(X, W_xf) + tf.matmul(H, W_hf) + b_f)
        O = tf.nn.sigmoid(tf.matmul(X, W_xo) + tf.matmul(H, W_ho) + b_o)
        C_tilda = tf.nn.tanh(tf.matmul(X, W_xc) + tf.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * tf.nn.tanh(C)
        Y = tf.matmul(H, W_hy) + b_y
        outputs.append(Y)
    return (outputs, H, C)

### 训练模型
下面我们开始训练模型。我们假定谱写歌词的前缀分别为“分开”、“不分开”和“战争中部队”。这里采用的是相邻批量采样实验门控循环单元谱写歌词。

In [8]:
slim = tf.contrib.slim

learning_rate = 1e-1
max_steps = 10000
batch_size = 32
train_loss = 0.0
train_acc = 0.0
is_random_iter= True
epochs= 50
num_steps= 35 
learning_rate= 1e-2
batch_size= 32
is_lstm = True

#训练
print hidden_dim
num_inputs = num_outputs = vocab_size
input_placeholder = tf.placeholder(tf.float32, [num_steps, None, num_inputs])
state_h_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])
print 'here'

state_c_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])
gt_placeholder = tf.placeholder(tf.int64, [num_steps, None, 1])

if is_lstm:
    # 当RNN使用LSTM时才会用到，这里可以忽略。
    outputs, state_h, state_c = lstm_rnn(input_placeholder, state_h_placeholder, state_c_placeholder, params)
else:
    outputs, state_h = rnn(input_placeholder, state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)

loss = tf.losses.sparse_softmax_cross_entropy(logits=outputs,  labels=tf.reshape(gt_placeholder, (num_steps*batch_size, 1)))

var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name
op = tf.train.AdamOptimizer(learning_rate)

gradients = tf.gradients(loss, params)

#process gradients
clipped_gradients, norm = tf.clip_by_global_norm(gradients, 1)

train_op = op.apply_gradients(zip(clipped_gradients, params))

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

if is_random_iter:
    data_iter = data_iter_random
else:
    data_iter = data_iter_consecutive
for e in range(0, epochs):
    # 如使用相邻批量采样，在同一个epoch中，隐含变量只需要在该epoch开始的时候初始化。
    if not is_random_iter:
        state_h_init = np.zeros(shape=(batch_size, hidden_dim))
        if is_lstm:
            # 当RNN使用LSTM时才会用到，这里可以忽略。
            state_c_init = np.zeros((batch_size, hidden_dim))
    train_loss, num_examples = 0, 0

    for data, label in data_iter(corpus_indices, batch_size, num_steps):
        # 如使用随机批量采样，处理每个随机小批量前都需要初始化隐含变量。
        if is_random_iter:
            state_h_init = np.zeros(shape=(batch_size, hidden_dim))
            if is_lstm:
                # 当RNN使用LSTM时才会用到，这里可以忽略。
                state_c_init = np.zeros((batch_size, hidden_dim))
                feed_dict = {input_placeholder: get_inputs(data), state_h_placeholder: state_h_init, state_c_placeholder: state_c_init, gt_placeholder: np.expand_dims(label.T, axis=-1)}
                loss_, state_h_, state_c_, _ = sess.run([loss, state_h, state_c, train_op], feed_dict=feed_dict)
                state_h_init = state_h_
                state_c_init = state_c_
            else:
                feed_dict = {input_placeholder: get_inputs(data), state_h_placeholder: state_h_init, gt_placeholder: np.expand_dims(label.T, axis=-1)}
                loss_, state_h_, _ = sess.run([loss, state_h, train_op], feed_dict=feed_dict)
                state_h_init = state_h_

        print np.exp(loss_)

256
here
rnn/weights_xi
rnn/weights_hi
rnn/bias_i
rnn/weights_xf
rnn/weights_hf
rnn/bias_f
rnn/weights_xo
rnn/weights_ho
rnn/bias_o
rnn/weights_xc
rnn/weights_hc
rnn/bias_c
rnn/weights_y
rnn/bias_y
1465.0054
1446.6422
1336.6493
579.2244
629.6254
568.9277
491.1674
485.17874
501.9527
456.99716
521.6606
492.75812
446.82953
537.7353
418.52774
394.7151
386.33472
314.07187
353.33072
399.35757
306.4753
395.93472
316.72934
395.2645
343.06134
310.43784
360.92493
358.66595
382.51663
362.1812
349.73572
317.208
366.74347
301.0899
296.95175
335.08267
302.10104
285.61328
318.74973
337.79404
312.68756
332.42575
345.10153
267.1354
294.80148
326.2878
294.47723
306.24362
324.6005
255.1995
299.3433
271.00662
276.6562
299.6069
252.93404
275.5353
277.9437
277.24637
298.81192
281.11435
263.84335
259.11795
260.08115
269.55942
264.33408
230.79706
233.93735
235.9854
242.55965
239.12659
202.14499
205.45868
223.99442
216.43933
196.0131
205.93192
227.01497
220.35992
180.69107
205.53893
231.59842
230.21628
192.904

1.044435
1.0503662
1.0454395
1.052401
1.0617268
1.0606318
1.0443627
1.0615921
1.055911
1.0452732
1.0589021
1.0666507
1.0601906
1.0733757
1.0527297
1.0531968
1.0344459
1.0609926
1.039907
1.0649062
1.0403237
1.0701368
1.0602999
1.0431423
1.0535117
1.06745
1.0511435
1.0586345
1.0675024
1.0696015
1.0564175
1.0650334


In [11]:
epochs = 200
num_steps = 35
learning_rate = 1e-4
batch_size = 32


seq1 = '分开'.decode('utf-8')
seq2 = '不分开'.decode('utf-8')
seq3 = '战争中部队'.decode('utf-8')
seqs = [seq1, seq2, seq3]
print seq1

分开


In [12]:
我们先采用随机批量采样实验循环神经网络谱写歌词。我们假定谱写歌词的前缀分别为“分开”、“不分开”和“战争中部队”

SyntaxError: invalid syntax (<ipython-input-12-66bfb44cf4d5>, line 1)

In [13]:
pred_len = 100
test_num_steps = 1
# 预测
num_inputs = num_outputs = vocab_size    
test_input_placeholder = tf.placeholder(tf.float32, [test_num_steps, None, num_inputs])
test_state_h_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])
test_state_c_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])

if is_lstm:
    # 当RNN使用LSTM时才会用到，这里可以忽略。
    outputs, state_h, state_c = lstm_rnn(test_input_placeholder, test_state_h_placeholder, test_state_c_placeholder, params)
else:
    outputs, state_h = rnn(test_input_placeholder, test_state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)


var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name


for seq in seqs:
    prefix = seq

    prefix = prefix.lower()
    test_state_h_init = np.zeros((1, hidden_dim))
    if is_lstm:
        # 当RNN使用LSTM时才会用到，这里可以忽略。
        test_state_c_init = np.zeros((1, hidden_dim))
    output_seq = [char_to_idx[prefix[0]]]
    for i in range(pred_len + len(prefix)):
        X = np.array([output_seq[-1]])
        # 在序列中循环迭代隐含变量。
        if is_lstm:
            # 当RNN使用LSTM时才会用到，这里可以忽略。
            Y, state_h_, state_c_ = sess.run([outputs, state_h, state_c], feed_dict={test_input_placeholder: get_inputs(X), test_state_c_placeholder: test_state_c_init, test_state_h_placeholder: test_state_h_init})
            test_state_h_init = state_c_
        else:
            Y, state_h_ = sess.run([outputs, state_h], feed_dict={test_input_placeholder: get_inputs(X), test_state_h_placeholder: test_state_h_init})
        test_state_h_init = state_h_
        if i < len(prefix)-1:
            next_input = char_to_idx[prefix[i+1]]
        else:
            next_input = np.argmax(Y[0])
        output_seq.append(next_input)
    print ''.join([idx_to_char[i] for i in output_seq])

    print()

rnn/weights_xi
rnn/weights_hi
rnn/bias_i
rnn/weights_xf
rnn/weights_hf
rnn/bias_f
rnn/weights_xo
rnn/weights_ho
rnn/bias_o
rnn/weights_xc
rnn/weights_hc
rnn/bias_c
rnn/weights_y
rnn/bias_y
分开始可爱女人 谁肯安慰 我们 半兽人 为什么我只剩下枪生 眼泪水在我有一条热 单已 我都没有一切 没人在人在人在人在人在人在战壕里没人帮着你会没人在屋檐走在弹唱 小铁步一步往 爬满的飞 一枚铜币的军河挡 
()
不分开的天堂景象 没有你甘会听 连咪都ㄟ 但是因 我给你的路 那些事 你慢慢慢去没人在美 我不好慢 我们将会分开 没人 离开始轮有 白墙黑口被象 你拿 告诉我们一起毕业的学校 反射出世事看你 连咪都提 我们一
()
战争中部队  谁肯定 没法挑剔它 漫天黄沙凉过 塞北的怀念 他们 半兽人 你叫我也许就快实现在我以让生命就这样的过去 试着黑色幽默 那首来没人在人在人在人在人在你在我有你ㄟ鱼 单影 有我害怕你心疼的我都没有错亏我
()
